# PREDICTION DES RETARDS DE VOL


## 1. IMPORT DES LIBRAIRIES


In [119]:
import pandas as pd
from os.path import join
from sklearn.model_selection import train_test_split

## 2. CHARGEMENT DES DONNÉES


Les données sont réparties en 12 fichiers csv, un par mois de l'année 2016.  
Chaque fichier comporte plus de 470000 lignes. Cela représente un total de plus de 5,5 millions de lignes. Il nous faut donc trouver une stratégie pour épargner notre mémoire.


### 2.1. Exploration d'un unique fichier

Dans ce cadre, nous décidons de commencer par analyser un fichier pour identifier les optimisations possibles afin de mes appliquer à tous les autres fichiers.


In [120]:
raw_data_path = join("..", "data", "raw")

jan_data_path = join(raw_data_path, "2016_01.csv")

In [121]:
df_jan = pd.read_csv(jan_data_path, on_bad_lines="warn", low_memory=False)
df_jan.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 64
0,2016,1,1,6,3,2016-01-06,AA,19805,AA,N4YBAA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1057.0,-3.0,0.0,0.0,-1.0,1100-1159,15.0,1112.0,1424.0,8.0,1438,1432.0,-6.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,158.0,155.0,132.0,1.0,986.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1,1,7,4,2016-01-07,AA,19805,AA,N434AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1056.0,-4.0,0.0,0.0,-1.0,1100-1159,14.0,1110.0,1416.0,10.0,1438,1426.0,-12.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,158.0,150.0,126.0,1.0,986.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,1,1,8,5,2016-01-08,AA,19805,AA,N541AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1055.0,-5.0,0.0,0.0,-1.0,1100-1159,21.0,1116.0,1431.0,14.0,1438,1445.0,7.0,7.0,0.0,0.0,1400-1459,0.0,NaN,0.0,158.0,170.0,135.0,1.0,986.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,1,1,9,6,2016-01-09,AA,19805,AA,N489AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1102.0,2.0,2.0,0.0,0.0,1100-1159,13.0,1115.0,1424.0,9.0,1438,1433.0,-5.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,158.0,151.0,129.0,1.0,986.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,1,1,10,7,2016-01-10,AA,19805,AA,N439AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1240.0,100.0,100.0,1.0,6.0,1100-1159,20.0,1300.0,1617.0,14.0,1438,1631.0,113.0,113.0,1.0,7.0,1400-1459,0.0,NaN,0.0,158.0,171.0,137.0,1.0,986.0,4,0.0,0.0,47.0,0.0,66.0,NaN,NaN,NaN,NaN


In [122]:
columns = df_jan.columns
columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'UNIQUE_CARRIER', 'AIRLINE_ID', 'CARRIER', 'TAIL_NUM', 'FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID',
       'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
       'ORIGIN_STATE_NM', 'ORIGIN_WAC', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15',
       'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE',
       'DISTANCE_GROUP', 'CARRIER_DELAY', 'WEATHER_DELAY

#### 2.1.1 Détermination de la cible

Nous avons identifié plusieurs candidats pour la cible : ARR_DELAY, ARR_DELAY_NEW, ARR_DEL15 et ARR_DELAY_GROUP


In [123]:
potential_targets = ["ARR_DELAY", "ARR_DELAY_NEW", "ARR_DEL15", "ARR_DELAY_GROUP"]

for target in potential_targets:
    print(f"{target}: Nombre de Valeurs Uniques")
    print("-" * 50)
    unique_count = df_jan[target].dropna().nunique()
    print(unique_count)
    if unique_count == 2:
        print(df_jan[target].dropna().unique())
    print("-" * 50)

ARR_DELAY: Nombre de Valeurs Uniques
--------------------------------------------------
735
--------------------------------------------------
ARR_DELAY_NEW: Nombre de Valeurs Uniques
--------------------------------------------------
660
--------------------------------------------------
ARR_DEL15: Nombre de Valeurs Uniques
--------------------------------------------------
2
[0. 1.]
--------------------------------------------------
ARR_DELAY_GROUP: Nombre de Valeurs Uniques
--------------------------------------------------
15
--------------------------------------------------


Nous constatons que seuls la colonne `ARR_DEL15` est sous forme binaire (en retard ou à l'heure) conformément à ce que nous souhaitons prédire. Les autres colonnes sont redondantes et peuvent donc être supprimées.


In [124]:
target = "ARR_DEL15"
redundant_targets = [col for col in potential_targets if col is not target]

print(redundant_targets)

['ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DELAY_GROUP']


Analysons cette colonne cible :


In [125]:
df_jan[target].info()

<class 'pandas.core.series.Series'>
RangeIndex: 445827 entries, 0 to 445826
Series name: ARR_DEL15
Non-Null Count   Dtype  
--------------   -----  
433298 non-null  float64
dtypes: float64(1)
memory usage: 3.4 MB


Nous constatons que le type de la colonne n'est pas adapté. Il faudrait le transformer en bouléen.


In [126]:
target_na_count = df_jan[target].isna().sum()
print(f"{target_na_count} lignes vide pour la cible {target}")

12529 lignes vide pour la cible ARR_DEL15


In [127]:
for col in redundant_targets:
    na_count = df_jan[col].isna().sum()
    print(f"{na_count} lignes vide pour la colonne {col}")

12529 lignes vide pour la colonne ARR_DELAY
12529 lignes vide pour la colonne ARR_DELAY_NEW
12529 lignes vide pour la colonne ARR_DELAY_GROUP


In [128]:
all_na_mask = df_jan[potential_targets].isna().all(axis=1)
all_na_count = all_na_mask.sum()

print(f"Lignes où TOUTES les colonnes sont NA : {all_na_count}")

Lignes où TOUTES les colonnes sont NA : 12529


Nous constatons que plusieurs milliers de lignes n'ont pas de valeurs cible. L'exploration des potentielles colonnes cibles précédemment écartées nous indique qu'il ne serait pas possible de reconstituer les valeurs manquantes avec ces autres colonnes.  
Il semble donc indispensable de les supprimer car ces lignes ne peuvent pas permettre d'entraîner le futur modèle.


#### 2.1.2. Identifications des colonnes aux valeurs manquantes


In [129]:
na_percentages = (df_jan.isna().sum() / len(df_jan)) * 100

high_na_cols = na_percentages[na_percentages > 50].index.tolist()

print("Colonnes avec plus de 50% de valeurs manquantes :")
for col in high_na_cols:
    print(f"- {col}: {na_percentages[col]:.2f}%")

print(f"\nListe des colonnes ({len(high_na_cols)}) :")
print(high_na_cols)

Colonnes avec plus de 50% de valeurs manquantes :
- CANCELLATION_CODE: 97.38%
- CARRIER_DELAY: 84.10%
- WEATHER_DELAY: 84.10%
- NAS_DELAY: 84.10%
- SECURITY_DELAY: 84.10%
- LATE_AIRCRAFT_DELAY: 84.10%
- FIRST_DEP_TIME: 99.43%
- TOTAL_ADD_GTIME: 99.43%
- LONGEST_ADD_GTIME: 99.43%
- Unnamed: 64: 100.00%

Liste des colonnes (10) :
['CANCELLATION_CODE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'FIRST_DEP_TIME', 'TOTAL_ADD_GTIME', 'LONGEST_ADD_GTIME', 'Unnamed: 64']


Nous avons identifié les colonnes pour lesquelles le taux de valeurs manquantes les rends caduques. Ces colonnes pourront être supprimées car il serait hasardeux d'essayer d'imputer les valeurs manquantes sur aussi peu de données.


#### 2.1.3. Identification des colonnes à risque de data leakage


Parmis les colonnes restantes certaines contiennent des informations qui influencent directement le résultat et qui ne seront normalement pas présente au moment de la prédiction :

- `DEP_TIME`, `ARR_TIME` : Heures réelles inconnues au moment de la prédiction
- `DEP_DELAY`, `DEP_DELAY_NEW`, `DEP_DEL15`, `DEP_DELAY_GROUP` : Retard au départ révèle directement le retard à l'arrivée
- `ARR_DELAY`, `ARR_DELAY_NEW`, `ARR_DEL15`, `ARR_DELAY_GROUP` : Variables cibles (à prédire)
- `WHEELS_OFF`, `WHEELS_ON`, `TAXI_OUT`, `TAXI_IN` : Données post-départ
- `ACTUAL_ELAPSED_TIME`, `AIR_TIME` : Durées réelles inconnues a priori
- `CANCELLED` et `DIVERTED` : Statuts du vol inconnus a priori

Ces colonnes doivent donc être supprimées.


In [130]:
data_leakage_cols = [
    "DEP_TIME",
    "ARR_TIME",
    "DEP_DELAY",
    "DEP_DELAY_NEW",
    "DEP_DEL15",
    "DEP_DELAY_GROUP",
    "ARR_DELAY",
    "ARR_DELAY_NEW",
    "ARR_DELAY_GROUP",
    "WHEELS_OFF",
    "WHEELS_ON",
    "TAXI_OUT",
    "TAXI_IN",
    "ACTUAL_ELAPSED_TIME",
    "AIR_TIME",
    "CANCELLED",
    "DIVERTED",
]

#### 2.1.4. Identification des colonnes redondantes


Parmi les colonnes restantes, nous identifions plusieurs colonnes redondantes entre elles :

- `AIRLINE_ID`, `UNIQUE_CARRIER` et `CARRIER` représentent toutes trois une façon d'identifier les compagnies aériennes. Nous ne conserverons que `UNIQUE_CARRIER`.
- Parmis les identifiants multiples pour l'origine et la destination, les codes IATA présents dans `ORIGIN` et `DEST` sont suffisament précis pour identifier à eux seuls l'aéroport d'origine et celui de destination.
- `ORIGIN_STATE_NM` et `ORIGIN_STATE_ABR` sont clairement redondants, dans notre cas l'abréviation seule suffira (`ORIGIN_STATE_ABR`).
- De la même façon, `DEST_STATE_NM` et `DEST_STATE_ABR` sont redondants. Nous ne conserverons que `DEST_STATE_ABR`
- `DISTANCE` et `DISTANCE_GROUP` sont redondantes. Nous choisissons de conserver `DISTANCE` car il s'agit d'une valeur continue, plus facile à interpréter.


Les colonnes redondantes n'apportant pas plus d'information, pourront être supprimées.


In [131]:
redundant_cols = [
    # Airline/Carrier related
    "AIRLINE_ID",
    "CARRIER",
    # Origin related
    "ORIGIN_AIRPORT_ID",
    "ORIGIN_AIRPORT_SEQ_ID",
    "ORIGIN_CITY_MARKET_ID",
    "ORIGIN_STATE_FIPS",
    "ORIGIN_STATE_NM",
    "ORIGIN_WAC",
    # Destination related
    "DEST_AIRPORT_ID",
    "DEST_AIRPORT_SEQ_ID",
    "DEST_CITY_MARKET_ID",
    "DEST_STATE_FIPS",
    "DEST_STATE_NM",
    "DEST_WAC",
    # Distance related
    "DISTANCE_GROUP",
]

#### 2.1.5. Analyse des colonnes temporelles


Parmi les colonnes temporelles (`YEAR`, `QUARTER`, `MONTH`, `DAY_OF_MONTH`,`DAY_OF_WEEK` et `FL_DATE`), il existe une certaine redondance.  
`FL_DATE` comporte toute l'information temporelle nécessaire en une seule colonne. Cependant, du point de vue du machine learning, il est plus facile de manipuler des valeurs numérique plutôt que des dates. Plusieurs stratégies s'offrent à nous :

- Tout conserver, mais cela ne semble pas très efficient,
- Ne conserver que `FL_DATE` dans notre future base de données et extraire les données temporelle en prétraitement
- Ne conserver que les données segmentées dans la base de données afin de les fournir directement lors de l'entrainement du modèle.


Nous choisirons la dernière méthode pour faciliter l'entraînement du modèle. Nous prenons églament le parti de supprimer la colonne `YEAR` car toutes nos données ne concernent que l'année 2016, cette colonne n'apporte donc aucune plus value à l'entraînement du modèle.


In [132]:
removable_temporal_cols = ["FL_DATE", "YEAR"]

#### 2.1.6 Cas de la colonne `FLIGHTS`


In [133]:
df_jan["FLIGHTS"].value_counts()

FLIGHTS
1.0    445827
Name: count, dtype: int64

Nous constatons que cette colonne, qui n'est d'ailleurs pas décrite, ne comprend qu'une valeur unique. Elle n'est donc pas nécessaire à la suite de l'analyse.


In [134]:
useless_cols = ["FLIGHTS"]

### 2.2. Application du traitement à tous les fichiers `.csv`


Maintenant que nous avons identifié toutes les colonnes à supprimer, nous allons pouvoir appliquer ce traitement à tous les fichiers fournis afin de les alléger et de pouvoir les traiter dans leur ensemble.


In [135]:
cols_to_drop = (
    redundant_targets
    + high_na_cols
    + data_leakage_cols
    + redundant_cols
    + removable_temporal_cols
    + useless_cols
)
cols_to_keep = [col for col in columns if col not in cols_to_drop]
len(cols_to_keep)

20

#### 2.2.1. Cas des données d'avril

Le fichier contenant les données du mois d'avril 2016 semble plus corrompu que les autres. Nous lui appliquerons donc un traitement particulier.


In [136]:
ap_data_path = join(raw_data_path, "2016_04.csv")
df_ap_clean = pd.read_csv(
    ap_data_path, on_bad_lines="warn", low_memory=False, usecols=cols_to_keep
)
df_ap_clean.head()

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME_BLK,CRS_ARR_TIME,ARR_DEL15,ARR_TIME_BLK,CRS_ELAPSED_TIME,DISTANCE
0,2,4,3,7,DL,N915DN,759,MSP,"Minneapolis, MN",MN,GEG,"Spokane, WA",WA,2000,2000-2059,2116,0.0,2100-2159,196.0,1175.0
1,2,4,3,7,DL,N3755D,760,ATL,"Atlanta, GA",GA,SMF,"Sacramento, CA",CA,0940,0900-0959,1156,0.0,1100-1159,316.0,2092.0
2,2,4,3,7,DL,N3755D,760,SMF,"Sacramento, CA",CA,ATL,"Atlanta, GA",GA,1248,1200-1259,2013,0.0,2000-2059,265.0,2092.0
3,2,4,3,7,DL,N325US,761,LAS,"Las Vegas, NV",NV,CVG,"Cincinnati, OH",KY,2345,2300-2359,0630,0.0,0600-0659,225.0,1678.0
4,2,4,3,7,DL,N366NB,762,ATL,"Atlanta, GA",GA,MDW,"Chicago, IL",IL,1647,1600-1659,1745,0.0,1700-1759,118.0,591.0


In [137]:
df_ap_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479956 entries, 0 to 479955
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   QUARTER           479956 non-null  object 
 1   MONTH             479956 non-null  int64  
 2   DAY_OF_MONTH      479956 non-null  object 
 3   DAY_OF_WEEK       479956 non-null  object 
 4   UNIQUE_CARRIER    479956 non-null  object 
 5   TAIL_NUM          479191 non-null  object 
 6   FL_NUM            479956 non-null  object 
 7   ORIGIN            479956 non-null  object 
 8   ORIGIN_CITY_NAME  479956 non-null  object 
 9   ORIGIN_STATE_ABR  479956 non-null  object 
 10  DEST              479955 non-null  object 
 11  DEST_CITY_NAME    479956 non-null  object 
 12  DEST_STATE_ABR    479955 non-null  object 
 13  CRS_DEP_TIME      479956 non-null  object 
 14  DEP_TIME_BLK      479954 non-null  object 
 15  CRS_ARR_TIME      479953 non-null  object 
 16  ARR_DEL15         47

Nous constatons rapidement que des colonnes qui devraient être de type int ou float (comme pour le fichier de janvier) sont de type object, ce qui indique probablement un type mixte pour ces colonnes. Analysons ces colonnes problématiques.


In [138]:
cols_to_check = [
    "QUARTER",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "FL_NUM",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "ARR_TIME_BLK",
]

for col in cols_to_check:
    print("-" * 50)
    print(col)
    print("-" * 50)
    print(df_ap_clean[col].value_counts())

--------------------------------------------------
QUARTER
--------------------------------------------------
QUARTER
2     362833
1     117122
EV         1
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_MONTH
--------------------------------------------------
DAY_OF_MONTH
25    18089
15    17895
29    17791
22    17762
21    17606
28    17398
27    17249
18    17077
20    17059
24    16783
14    16641
26    16330
6     16285
19    16254
17    15656
3     15608
23    15571
13    15188
8     15041
30    14990
1     14880
12    14874
4     14800
7     14741
11    14619
16    14581
5     14326
2     14002
10    13899
9     12570
31     4390
EV        1
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_WEEK
--------------------------------------------------
DAY_OF_WEEK
5         76518
2         70535
4         69843
3         69261
1         66119
6         64559
7         63120
N707EV        1
Name: count, dtype: int64

Nous constatons certaines valeurs aberrantes parmi les valeurs uniques de certaines colonnes. Une seule ligne semble concernée à chaque fois. Regardons s'il s'agit toujours de la même ligne.

In [139]:
ev_rows = df_ap_clean[df_ap_clean["QUARTER"] == "EV"]

In [140]:
df_ap_clean = df_ap_clean.drop(index=461814)
df_ap_clean[df_ap_clean["QUARTER"] == "EV"]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME_BLK,CRS_ARR_TIME,ARR_DEL15,ARR_TIME_BLK,CRS_ELAPSED_TIME,DISTANCE


In [141]:
cols_to_check = [
    "QUARTER",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "FL_NUM",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "ARR_TIME_BLK",
]

for col in cols_to_check:
    print("-" * 50)
    print(col)
    print("-" * 50)
    print(df_ap_clean[col].value_counts())

--------------------------------------------------
QUARTER
--------------------------------------------------
QUARTER
2    362833
1    117122
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_MONTH
--------------------------------------------------
DAY_OF_MONTH
25    18089
15    17895
29    17791
22    17762
21    17606
28    17398
27    17249
18    17077
20    17059
24    16783
14    16641
26    16330
6     16285
19    16254
17    15656
3     15608
23    15571
13    15188
8     15041
30    14990
1     14880
12    14874
4     14800
7     14741
11    14619
16    14581
5     14326
2     14002
10    13899
9     12570
31     4390
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_WEEK
--------------------------------------------------
DAY_OF_WEEK
5    76518
2    70535
4    69843
3    69261
1    66119
6    64559
7    63120
Name: count, dtype: int64
--------------------------------------------------
FL_NUM
-------------------

Nous constatons qu'étrangement, ce fichier qui est censé ne concerner que le mois d'avril contient un nombre significatif des valeurs de `QUARTER` à 1 au lieu de 2.

**H0 (Hypothèse 0) :** Certaines entrées du mois de mars ont été incluses dans les données du mois d'avril

**H1 (Hypothèse 1) :** Le fichier d'avril contient des données corrompues avec des valeurs non-numériques dans des colonnes qui devraient être numériques

**H2 (Hypothèse 2) :** Il existe une incohérence dans l'encodage ou le format des données du fichier d'avril par rapport aux autres mois

**H3 (Hypothèse 3) :** Certaines colonnes du fichier d'avril ont été mal parsées lors de l'extraction ou de la création du fichier CSV

**H4 (Hypothèse 4) :** Le fichier d'avril contient des caractères spéciaux ou des délimiteurs incorrects qui causent une mauvaise lecture des données


In [142]:
# Créer une condition de cohérence
coherence_check = ((df_ap_clean["MONTH"] == "3") & (df_ap_clean["QUARTER"] == "1")) | (
    (df_ap_clean["MONTH"] == "4") & (df_ap_clean["QUARTER"] == "2")
)

# Identifier les incohérences
incoherent_rows = df_ap_clean[~coherence_check & df_ap_clean["MONTH"].isin(["3", "4"])]

print(f"Nombre de lignes incohérentes: {len(incoherent_rows)}")
if len(incoherent_rows) > 0:
    print("Échantillon des incohérences:")
    print(incoherent_rows[["MONTH", "QUARTER"]].head())

Nombre de lignes incohérentes: 0


L'hypothèse selon laquelle des lignes du mois de mars sont incluses dans les données du mois d'avril se vérifie dans la mesure où il existe une cohérence parfaite entre le mois et le semestre.


In [143]:
df_ap_clean["MONTH"].value_counts()

MONTH
4    362833
3    117122
Name: count, dtype: int64

In [144]:
def analyze_object_columns(df, verbose=True):
    """Analyse les colonnes object pour identifier celles qui devraient être numériques"""

    object_cols = df.select_dtypes(include=["object"]).columns
    problematic_cols = {}

    if verbose:
        print("=== ANALYSE DES COLONNES OBJECT ===")
        print(f"Colonnes object trouvées: {len(object_cols)}")
        print("-" * 50)

    for col in object_cols:
        if verbose:
            print(f"\n📊 Analyse de la colonne: {col}")

        # Compter les valeurs uniques
        unique_values = df[col].dropna().unique()

        # Identifier les valeurs qui ne sont pas numériques
        non_numeric_values = []
        numeric_count = 0

        for val in unique_values:
            try:
                float(val)
                numeric_count += 1
            except (ValueError, TypeError):
                non_numeric_values.append(val)

        if non_numeric_values and numeric_count > 0:
            # Cette colonne contient un mélange de valeurs numériques et non-numériques
            total_values = len(df[col].dropna())
            non_numeric_rows = df[col].isin(non_numeric_values).sum()
            percentage = (non_numeric_rows / total_values) * 100

            problematic_cols[col] = {
                "non_numeric_values": non_numeric_values,
                "non_numeric_count": non_numeric_rows,
                "total_count": total_values,
                "percentage": percentage,
            }

            if verbose:
                print(f"  ⚠️  Colonne mixte détectée!")
                print(f"  Valeurs non-numériques: {non_numeric_values}")
                print(
                    f"  Lignes affectées: {non_numeric_rows}/{total_values} ({percentage:.2f}%)"
                )

        elif non_numeric_values and numeric_count == 0:
            if verbose:
                print(f"  ✅ Colonne entièrement non-numérique (normal)")
        else:
            if verbose:
                print(f"  ✅ Colonne entièrement numérique (peut être convertie)")

    return problematic_cols


pb = analyze_object_columns(df_ap_clean)
print(pb)

=== ANALYSE DES COLONNES OBJECT ===
Colonnes object trouvées: 16
--------------------------------------------------

📊 Analyse de la colonne: QUARTER
  ✅ Colonne entièrement numérique (peut être convertie)

📊 Analyse de la colonne: DAY_OF_MONTH


  ✅ Colonne entièrement numérique (peut être convertie)

📊 Analyse de la colonne: DAY_OF_WEEK
  ✅ Colonne entièrement numérique (peut être convertie)

📊 Analyse de la colonne: UNIQUE_CARRIER
  ✅ Colonne entièrement non-numérique (normal)

📊 Analyse de la colonne: TAIL_NUM
  ✅ Colonne entièrement non-numérique (normal)

📊 Analyse de la colonne: FL_NUM
  ⚠️  Colonne mixte détectée!
  Valeurs non-numériques: ['582700-1759']
  Lignes affectées: 1/479955 (0.00%)

📊 Analyse de la colonne: ORIGIN
  ⚠️  Colonne mixte détectée!
  Valeurs non-numériques: ['MSP', 'ATL', 'SMF', 'LAS', 'MDW', 'SLC', 'MKE', 'AUS', 'CLT', 'PBI', 'PDX', 'SAT', 'MIA', 'MSY', 'FLL', 'DEN', 'DCA', 'RNO', 'RSW', 'CVG', 'MCO', 'GSO', 'PHL', 'EWR', 'ORF', 'BOS', 'IAH', 'TPA', 'ORD', 'SAN', 'SNA', 'LAX', 'ANC', 'SEA', 'PHX', 'ECP', 'LGA', 'DTW', 'OAK', 'MCI', 'HNL', 'CHS', 'MSN', 'MEM', 'ELP', 'SFO', 'PIT', 'DFW', 'JAN', 'BWI', 'GRR', 'MYR', 'FNT', 'FSD', 'BNA', 'JAX', 'SJU', 'RDU', 'STT', 'STL', 'IND', 'ALB', 'BDL', 'GPT', 

In [145]:
jan_float_cols = df_jan.select_dtypes(include=["float64", "float32"]).columns
ap_float_cols = df_ap_clean.select_dtypes(include=["float64", "float32"]).columns
print(len(jan_float_cols))
print(len(ap_float_cols))

30
3


In [146]:
df_ap_clean["QUARTER"]

0         2
1         2
2         2
3         2
4         2
         ..
479951    1
479952    1
479953    1
479954    1
479955    1
Name: QUARTER, Length: 479955, dtype: object

In [147]:
# Rechercher les lignes où QUARTER contient "EV"
ev_rows = df_ap_clean[df_ap_clean["QUARTER"] == "EV"]

print(f"Nombre de lignes avec QUARTER = 'EV': {len(ev_rows)}")

if len(ev_rows) > 0:
    print("\nÉchantillon des lignes avec QUARTER = 'EV':")
    print(ev_rows.head())

    print("\nInformations sur ces lignes:")
    print(ev_rows.info())

    print("\nValeurs des autres colonnes pour ces lignes:")
    print(ev_rows[["MONTH", "QUARTER", "DAY_OF_MONTH", "DAY_OF_WEEK"]].head(10))
else:
    print("Aucune ligne trouvée avec QUARTER = 'EV'")

Nombre de lignes avec QUARTER = 'EV': 0
Aucune ligne trouvée avec QUARTER = 'EV'


#### 2.2.2. Nettoyage de tous les fichiers


In [ ]:
pd.set_option("display.max_rows", 65)
pd.set_option("display.max_columns", 65)

df_year = []
total_lines_count = 0


def clean_target_column(df, target_col="ARR_DEL15"):
    """Nettoie et convertit la colonne cible"""
    print("-" * 50)
    print(f"----- Nettoyage de la cible ({target_col}) -----")
    print("-" * 50)

    df_clean = df.copy()

    df_clean = df_clean.dropna(subset=[target_col])

    current_dtype = df_clean[target_col].dtype
    print(f"Type actuel de {target_col}: {current_dtype}")

    if current_dtype in ["float64", "float32"]:
        # Convertir float -> int -> bool
        df_clean[target_col] = df_clean[target_col].astype("int8").astype("bool")
    elif current_dtype in ["int64", "int32", "int8"]:
        # Convertir int -> bool
        df_clean[target_col] = df_clean[target_col].astype("bool")
    else:
        # Conversion directe pour les autres types
        df_clean[target_col] = df_clean[target_col].astype("bool")

    print("La cible a été nettoyée et son type transformé en bouléen ✅")
    return df_clean


def optimize_dtypes(df, verbose=True):
    """Optimise les types de données pour économiser la mémoire"""
    print("-" * 50)
    print("----- Optimisation des types -----")
    print("-" * 50)

    # Convertir float64/float32 vers int32
    float_cols = df.select_dtypes(include=["float64", "float32"]).columns
    for col in float_cols:
        df[col] = df[col].astype("int32")
        if verbose:
            print(f"{col} a été transformée en int32")

    # Convertir int64 vers int32
    int64_cols = df.select_dtypes(include=["int64"]).columns
    for col in int64_cols:
        df[col] = df[col].astype("int32")
        if verbose:
            print(f"{col} a été transformée en int32")

    return df


def delete_duplicates(df):
    """Supprime les doublons de ligne"""
    print("-" * 50)
    print("---- Gestion des doublons de ligne ----")
    print("-" * 50)

    duplicated_rows = df.duplicated()
    nb_duplicates = duplicated_rows.sum()

    if nb_duplicates > 0:
        print("⚠️ DOUBLONS DÉTECTÉS!")
        print(f"Nombre de lignes dupliquées: {nb_duplicates}")
        print(f"Pourcentage de doublons: {(nb_duplicates / len(df)) * 100:.2f}%")
        print(f"Lignes avant suppression: {len(df)}")

        df = df.drop_duplicates()

        print(f"Lignes après suppression: {len(df)}")
        print(f"Lignes supprimées: {nb_duplicates}")
    else:
        print("✅ Aucun doublon de ligne détecté")

    return df


for month in range(1, 13):
    data_path = join(raw_data_path, f"2016_{month:02}.csv")
    print("=" * 50)
    print(f"----- RAPPORT : {data_path} -----")
    print("=" * 50)
    try:
        month_df = pd.read_csv(
            data_path, on_bad_lines="warn", low_memory=False, usecols=cols_to_keep
        )
        if month == 4:
            month_df = month_df.dropna()

        month_df = clean_target_column(month_df)
        month_df = optimize_dtypes(month_df)
        month_df = delete_duplicates(month_df)

        df_year.append(month_df)
        lines_count = month_df.shape[0]
        total_lines_count += lines_count
        print(f"{lines_count} lines loaded from {data_path} ✅")
    except pd.errors.ParserError as err:
        print(f"ParserError for {data_path}: {err}")

print(f"For a total lines count of: {total_lines_count}")


----- RAPPORT : ../data/raw/2016_01.csv -----
--------------------------------------------------
----- Nettoyage de la cible (ARR_DEL15) -----
--------------------------------------------------
Type actuel de ARR_DEL15: float64
La cible a été nettoyée et son type transformé en bouléen ✅
--------------------------------------------------
----- Optimisation des types -----
--------------------------------------------------
CRS_ELAPSED_TIME a été transformée en int32
FLIGHTS a été transformée en int32
DISTANCE a été transformée en int32
QUARTER a été transformée en int32
MONTH a été transformée en int32
DAY_OF_MONTH a été transformée en int32
DAY_OF_WEEK a été transformée en int32
FL_NUM a été transformée en int32
CRS_DEP_TIME a été transformée en int32
CRS_ARR_TIME a été transformée en int32
--------------------------------------------------
---- Gestion des doublons de ligne ----
--------------------------------------------------
✅ Aucun doublon de ligne détecté
433298 lines loaded from

#### 2.2.3. Concaténation de toutes les données


In [ ]:
complete_df = pd.concat(df_year, ignore_index=True)

In [ ]:
df = complete_df.copy()
df = df.drop_duplicates()

Nous commentons les lignes suivantes pour éviter la génération du csv concaténé à chaque fois que nous lançons le notebook.


In [ ]:
# complete_data_path = join(raw_data_path, "complete.csv")
# df.to_csv(complete_data_path)

## 3. EXPLORATION DES DONNÉES


In [ ]:
origins = complete_df["ORIGIN"].unique().tolist()
dest = complete_df["DEST"].unique().tolist()

airports = pd.Series(origins + dest).unique().tolist()

print(f"Nombre total d'aéroports uniques: {len(airports)}")

Nombre total d'aéroports uniques: 312


In [ ]:
origin_states = complete_df["ORIGIN_STATE_ABR"].unique().tolist()
dest_states = complete_df["DEST_STATE_ABR"].unique().tolist()

states = pd.Series(origin_states + dest_states).unique().tolist()

print(f"Nombre total d'états uniques: {len(states)}")

Nombre total d'états uniques: 55


In [ ]:
origin_cities = complete_df["ORIGIN_CITY_NAME"].unique().tolist()
dest_cities = complete_df["ORIGIN_CITY_NAME"].unique().tolist()

cities = pd.Series(origin_cities + dest_cities).unique().tolist()

print(f"Nombre total de villes uniques: {len(cities)}")

Nombre total de villes uniques: 306


In [ ]:
airlines = complete_df["UNIQUE_CARRIER"].unique().tolist()

print(f"Nombre total de compagnies aériennes uniques: {len(airlines)}")

Nombre total de compagnies aériennes uniques: 12


## 4. VISUALISATIONS


## 5.MODÉLISATION
